<a href="https://colab.research.google.com/github/lujosylvia/Stress-Detection-Transformer/blob/main/transformer_experiments_WESAD_wrist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Research Project - Stress Detection

The goal of this project is to determine which signal, or combinations of signals, are most performant in detecting stress levels when used with an encoder-only Transformer model. I want to also determine if signals from the wrist are comparable to those from the chest. The equipment recording signals from the chest should be more accurate, but it would be nice to verify that those in the wrist are as useful so that wrist devices, which are more easily accessible and easier to wear, can be proven as reliable enough.

In [ ]:
import itertools
import pickle
import pandas as pd
import numpy as np
import torch.utils.data
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score

# custom modules
from dataset_builders import build_wrist_dataframe
from transformer import TransformerStressPredictor, time_series_split_train_test_validation


In [ ]:
if not torch.cuda.is_available():
    print("CUDA is not available. Please check your GPU setup.")
    Device = torch.device("cpu")
else:
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    Device = torch.device('cuda:0')

Device

Using GPU: NVIDIA A100-SXM4-40GB


device(type='cuda', index=0)

### Read Data Files
As part of the WESAD dataset, according to the README, for each subject, the correlating `.pkl` file includes all of the signals recorded in a synchronized manner and labelled. These files should be used when processing data.

### Load Wrist Data

In [ ]:
wrist_data = {}

subjects = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
for subject in subjects:
  wrist_data[f"S{subject}"] = build_wrist_dataframe(f"./WESAD/S{subject}/S{subject}.pkl")

wrist_data.keys()

dict_keys(['S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17'])

In [ ]:
wrist_data['S3'].head()

,ACC_1,ACC_2,ACC_3,BVP,EDA,TEMP,label
0,0.315923,0.100602,-0.039766,-1.434623,0.290443,2.625071,1
1,0.318006,0.105633,-0.038816,-1.427695,0.294175,2.624843,1
2,0.320086,0.110656,-0.037867,-1.420781,0.297903,2.624616,1
3,0.322162,0.115671,-0.036922,-1.413881,0.301627,2.624388,1
4,0.324236,0.120678,-0.035979,-1.406994,0.305347,2.624161,1


## How does model perform using EDA alone?

In [ ]:
%%time
clf = TransformerStressPredictor(input_dims=1, model_dim=32, nhead=4, n_encoder_layers=2, batch_size=4096).to(Device)
time_series_split_train_test_validation(clf, wrist_data, ['EDA'], epochs=50, _batch_size=4096)

 50/ 50 | Loss: 1.437

Acc= 0.97
F1= 0.98
Loss= 1.78


 50/ 50 | Loss: 0.919

Acc= 0.87
F1= 0.87
Loss= 1.08


 50/ 50 | Loss: 0.941

...

 50/ 50 | Loss: 2.080

Acc= 0.93
F1= 0.95
Loss= 2.26


 50/ 50 | Loss: 1.198

Acc= 0.68
F1= 0.55
Loss= 1.32
CPU times: user 17min 42s, sys: 36.1 s, total: 18min 18s
Wall time: 7min 6s


(0.8809815719140202,
 0.8674473105865325,
 1.5728448768658128,
 [1.9402508651837707,
  1.6655161926755682,
  1.7166017882409506,
  1.5157732848310843,
  1.7323278214898892,
  1.6071788032422774,
  1.5415655666729435,
  1.444934752304107,
  1.3959485699888319,
  1.3949774890206754,
  1.3315560332266614,
  1.371993872220628,
  1.288641770137474,
  1.2610849429038353,
  1.2297420469112694,
  1.2095934944227338,
  1.190096546546556,
  1.1695977232884616,
  1.1505293474765494,
  1.1659537492669187,
  1.2069457694888115,
  2.224781653145328,
  1.3258933972683735,
  1.2858906359761022,
  1.2802408544812351,
  1.2412061074282974,
  1.2394229304627515,
  1.2324520072434098,
  1.2245503721060231,
  1.223715468251612,
  1.2156713308067992,
  1.210869827948045,
  1.2099021488102153,
  1.206436089181807,
  1.1996244842885062,
  1.2029180327663198,
  1.200980915920809,
  1.201425054227002,
  1.20088775025215,
  1.1976433151285164,
  1.199162233446259,
  1.1993336547748186,
  1.1996236535487697,
  1.

## How does model perform using ACC alone?

In [ ]:
%%time
clf = TransformerStressPredictor(input_dims=3, model_dim=32, nhead=4, n_encoder_layers=2, batch_size=4096).to(Device)
time_series_split_train_test_validation(clf, wrist_data, ['ACC_1', 'ACC_2', 'ACC_3'], epochs=50, _batch_size=4096)

 50/ 50 | Loss: 2.251

Acc= 0.97
F1= 0.97
Loss= 2.91


 50/ 50 | Loss: 0.699

Acc= 0.78
F1= 0.72
Loss= 1.14


 50/ 50 | Loss: 1.631

...

 50/ 50 | Loss: 0.953

Acc= 0.60
F1= 0.55
Loss= 1.42


 50/ 50 | Loss: 1.142

Acc= 0.86
F1= 0.90
Loss= 2.01
CPU times: user 21min 7s, sys: 39.8 s, total: 21min 46s
Wall time: 10min 10s


(0.75905888378172,
 0.7308619372860232,
 1.9485062935223176,
 [6.725033282811637,
  4.758478751522489,
  3.125148069113493,
  2.824267670046538,
  2.6811252585612237,
  2.745018631219864,
  2.5840224837884307,
  2.5215022643096745,
  2.5236179237253964,
  2.566044035134837,
  2.6639943458139896,
  2.282333450508304,
  2.24439960392192,
  2.253519739722833,
  2.2044525537639856,
  2.192737893667072,
  2.1835777438245714,
  2.1326085191685706,
  2.153355361078866,
  2.029196161078289,
  2.101192946312949,
  2.041181433480233,
  2.135717835801188,
  1.7851587374461815,
  1.777861897367984,
  1.624427038099384,
  1.7016500235768035,
  1.6413803123286925,
  1.6132099639507942,
  1.577750660770107,
  1.71573089712183,
  1.5716975851682946,
  1.5413415353395976,
  1.5169462281046435,
  1.4212144324555993,
  1.4635650004493073,
  1.2600049437314738,
  1.5018453025841154,
  1.5874825585342478,
  1.4551690685620997,
  1.2893872811400797,
  1.2560784504166804,
  1.23122480296297,
  1.140093963476

## How does model perform using BVP alone?

In [ ]:
%%time
clf = TransformerStressPredictor(input_dims=1, model_dim=32, nhead=4, n_encoder_layers=2, batch_size=4096).to(Device)
time_series_split_train_test_validation(clf, wrist_data, ['BVP'], epochs=50, _batch_size=4096)

 50/ 50 | Loss: 3.428

Acc= 1.00
F1= 1.00
Loss= 3.77


 50/ 50 | Loss: 3.674

Acc= 1.00
F1= 1.00
Loss= 3.76


 50/ 50 | Loss: 3.112

...

 50/ 50 | Loss: 3.958

Acc= 1.00
F1= 1.00
Loss= 3.98


 50/ 50 | Loss: 3.259

Acc= 0.72
F1= 0.60
Loss= 3.29
CPU times: user 17min 41s, sys: 38 s, total: 18min 19s
Wall time: 7min 8s


(0.9436693050986843,
 0.9201200724693075,
 3.5958625540731015,
 [3.3827688805758953,
  3.373067483305931,
  3.39060315489769,
  3.3892535846680403,
  3.379457078874111,
  3.318564573302865,
  3.316854452714324,
  3.3169024735689163,
  3.3163149747997522,
  3.3150735646486282,
  3.3149075135588646,
  3.3142484817653894,
  3.315686719492078,
  3.315857984125614,
  3.3148504309356213,
  3.2870590705424547,
  3.287499899044633,
  3.2873369213193655,
  3.286992961540818,
  3.2723956387490034,
  3.271601004526019,
  3.2732886262238026,
  3.2733001429587603,
  3.2732022292912006,
  3.266443185508251,
  3.265443479642272,
  3.2656452115625143,
  3.265415655449033,
  3.265685673803091,
  3.262948302552104,
  3.2618065010756254,
  3.2620354536920786,
  3.2621075697243214,
  3.261742275208235,
  3.260531898587942,
  3.2607020251452923,
  3.2610202580690384,
  3.2604052163660526,
  3.2597514539957047,
  3.2595692053437233,
  3.259674109518528,
  3.2598170414566994,
  3.258616417646408,
  3.2591347

## How does BVP, ACC channel X, and ACC channel Z perform?

In [ ]:
%%time
clf = TransformerStressPredictor(input_dims=3, model_dim=32, nhead=4, n_encoder_layers=2, batch_size=4096).to(Device)
time_series_split_train_test_validation(clf, wrist_data, ['BVP', 'ACC_1', 'ACC_3'], epochs=50, _batch_size=4096)

 50/ 50 | Loss: 2.999

Acc= 0.91
F1= 0.95
Loss= 3.37


 50/ 50 | Loss: 3.015

Acc= 1.00
F1= 1.00
Loss= 3.12


 50/ 50 | Loss: 3.243

...

 50/ 50 | Loss: 3.607

Acc= 1.00
F1= 1.00
Loss= 3.78


 50/ 50 | Loss: 2.975

Acc= 0.80
F1= 0.71
Loss= 3.31
CPU times: user 21min 8s, sys: 39.1 s, total: 21min 47s
Wall time: 10min 25s


(0.9429840856067649,
 0.9336832089277711,
 3.3779649561984697,
 [4.018825785256922,
  3.855234581977129,
  3.7849749010056257,
  3.7336365524679422,
  3.69149374589324,
  3.683724472299218,
  3.680583544075489,
  3.5800601076334715,
  3.638388277962804,
  3.753575559705496,
  3.5328672491014004,
  3.372254876419902,
  3.3724395353347063,
  3.573417402803898,
  3.6560173612087965,
  3.3360922001302242,
  3.3161418233066797,
  3.218804556876421,
  3.231938587501645,
  3.251831825822592,
  3.336828526109457,
  3.3130438700318336,
  3.2759191766381264,
  3.218317160382867,
  3.2143656257539988,
  3.2108153868466616,
  3.1950209867209196,
  3.1531576924026012,
  3.1469768173992634,
  3.1749405656009912,
  3.1395028438419104,
  3.1324019841849804,
  3.130184354260564,
  3.1250248979777098,
  3.1209239717572927,
  3.115894950926304,
  3.0978651847690344,
  3.1946778297424316,
  3.309277905151248,
  3.1511290092021227,
  3.211094679310918,
  3.1194058377295732,
  3.0230752862989902,
  3.112071